# 02 — Renda vs Desempenho (ENEM 2023)

Este notebook analisa a relação entre **renda familiar (Q006)** e **desempenho** no ENEM 2023.

**Premissas:**
- O arquivo `../data/interim/enem_2023.parquet` já foi gerado.
- Colunas mínimas: `NOTA_MEDIA_5`, `Q006`, `Q006_ord`.
- Colunas opcionais: `Q006_rot`, `TP_ESCOLA_rot`.


In [ ]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

DATA_PATH = Path("../data/interim/enem_2023.parquet")
print("Lendo:", DATA_PATH)
df = pd.read_parquet(DATA_PATH)
print("Dimensões:", df.shape)

required = ["NOTA_MEDIA_5", "Q006", "Q006_ord"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise RuntimeError(f"Faltam colunas necessárias: {missing}.")

RENDA_ROT_COL = "Q006_rot" if "Q006_rot" in df.columns else None
TP_ESCOLA_ROT = "TP_ESCOLA_rot" if "TP_ESCOLA_rot" in df.columns else None

if "IN_TREINEIRO" in df.columns:
    df = df[df["IN_TREINEIRO"] == 0].copy()

df = df.dropna(subset=["NOTA_MEDIA_5", "Q006", "Q006_ord"]).copy()
df.head()

## 1) Distribuição de Renda (Q006) — Absoluta e Percentual

In [ ]:
# Frequência absoluta
plt.figure(figsize=(10, 6))
if RENDA_ROT_COL:
    counts = df[RENDA_ROT_COL].value_counts().sort_index()
    plt.barh(counts.index, counts.values)
    plt.title("Distribuição da Renda Familiar (Q006) — Frequência Absoluta")
    plt.xlabel("Número de Estudantes")
    plt.ylabel("Faixa de Renda (rótulo)")
else:
    counts = df["Q006"].value_counts().sort_index()
    plt.bar(counts.index, counts.values)
    plt.title("Distribuição da Renda Familiar (Q006) — Códigos")
    plt.xlabel("Q006 (códigos A..Z)")
    plt.ylabel("Número de Estudantes")
plt.tight_layout()
plt.show()

# Percentual
plt.figure(figsize=(10, 6))
if RENDA_ROT_COL:
    pct = (df[RENDA_ROT_COL].value_counts(normalize=True).sort_index() * 100)
    plt.barh(pct.index, pct.values)
    plt.title("Distribuição Percentual da Renda Familiar (Q006)")
    plt.xlabel("% de Estudantes")
    plt.ylabel("Faixa de Renda (rótulo)")
else:
    pct = (df["Q006"].value_counts(normalize=True).sort_index() * 100)
    plt.bar(pct.index, pct.values)
    plt.title("Distribuição Percentual da Renda Familiar (Q006)")
    plt.xlabel("% de Estudantes")
    plt.ylabel("Q006 (códigos A..Z)")
plt.tight_layout()
plt.show()

## 2) Tendência da Nota Média por Faixa de Renda (ordem ordinal)

In [ ]:
ord_df = df.groupby("Q006_ord", as_index=False)["NOTA_MEDIA_5"].mean().sort_values("Q006_ord")
plt.figure(figsize=(10, 5))
plt.plot(ord_df["Q006_ord"], ord_df["NOTA_MEDIA_5"], marker="o")
plt.title("Tendência da Nota Média por Faixa de Renda (Q006_ord)")
plt.xlabel("Ordem de Renda (A=1, B=2, ...)")
plt.ylabel("Nota Média (5 provas)")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

# Versão com rótulos, se disponível
if RENDA_ROT_COL:
    tmp_lab = df[[RENDA_ROT_COL, "Q006_ord"]].dropna().drop_duplicates().sort_values("Q006_ord")
    x = list(tmp_lab["Q006_ord"])
    labs = list(tmp_lab[RENDA_ROT_COL])
    means = df.groupby("Q006_ord")["NOTA_MEDIA_5"].mean().reindex(x)

    plt.figure(figsize=(12, 5))
    plt.plot(labs, means.values, marker="o")
    plt.title("Tendência da Nota Média por Faixa de Renda (rótulos)")
    plt.xlabel("Faixa de Renda (Q006_rot)")
    plt.ylabel("Nota Média (5 provas)")
    plt.xticks(rotation=45, ha="right")
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()
    plt.show()

## 3) Boxplot da Nota Média por Faixa de Renda

In [ ]:
if RENDA_ROT_COL:
    order = df[[RENDA_ROT_COL, "Q006_ord"]].drop_duplicates().sort_values("Q006_ord")
    groups = [df.loc[df[RENDA_ROT_COL]==lab, "NOTA_MEDIA_5"].dropna().values for lab in order[RENDA_ROT_COL]]
    labels = list(order[RENDA_ROT_COL])
else:
    order = sorted(df["Q006"].dropna().unique())
    groups = [df.loc[df["Q006"]==lab, "NOTA_MEDIA_5"].dropna().values for lab in order]
    labels = order

plt.figure(figsize=(12, 6))
plt.boxplot(groups, labels=labels, showfliers=False)
plt.title("Boxplot da Nota Média por Faixa de Renda (Q006)")
plt.xlabel("Faixa de Renda")
plt.ylabel("Nota Média (5 provas)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

## 4) Correlação e Dispersão (Q006_ord × NOTA_MEDIA_5)

In [ ]:
cor = df["Q006_ord"].corr(df["NOTA_MEDIA_5"])
print(f"Correlação (Q006_ord vs NOTA_MEDIA_5): {cor:.3f}")

sample = df.sample(min(50000, len(df)), random_state=42)
plt.figure(figsize=(7, 5))
plt.scatter(sample["Q006_ord"], sample["NOTA_MEDIA_5"], s=6, alpha=0.2)
plt.title(f"Dispersão: Renda (ord) x Nota Média — corr={cor:.3f}")
plt.xlabel("Q006_ord (A=1, B=2, ...)")
plt.ylabel("NOTA_MEDIA_5")
plt.grid(True, linestyle=":", alpha=0.5)
plt.tight_layout()
plt.show()

## 5) Observações para o relatório

- A distribuição de renda concentra-se nas faixas intermediárias; faixas altas são minoritárias.  
- A **nota média cresce** de forma monotônica com o aumento da renda (linha e boxplot).  
- A **correlação positiva** entre renda ordinal e nota média reforça esse padrão.  
- (Se estratificado por tipo de escola) a diferença tende a se manter, sugerindo que renda impacta independentemente do setor.
